This notebook loads FRAP data from both foci and a focus-free region of the ER, then plotse these data as individual recovery figures as well as a summary boxplot figure

In [1]:
# load the custom modules

# uncomment for debugging
%load_ext autoreload 
%autoreload 2 

import os, sys, inspect
import matplotlib.pylab as plt
import numpy as np
from pprint import pprint

# Add source code directory (src) to path to enable module import
curr_frame = inspect.getfile(inspect.currentframe())
curr_dir = os.path.dirname(os.path.abspath(curr_frame))
parent_dir = os.path.dirname(curr_dir)
module_dir = os.path.join(parent_dir, 'src')
os.sys.path.insert(0, module_dir)

import parse_trackmate as pt
import fraptools as frap


In [2]:
# Load data
import glob

# Path to raw FRAP data (TrackMate xml) and background data (json)
data_dir = '../data/raw/IRE1_foci_FRAP'
frap_subdir = 'FRAP_traces'
bkgnd_subdir = 'bkgnd_traces'

frap_dir = os.path.join(data_dir,frap_subdir)
bkgnd_dir = os.path.join(data_dir,bkgnd_subdir)

frap_files = sorted(glob.glob(os.path.join(frap_dir,'*.xml')))
bkgnd_files = sorted(glob.glob(os.path.join(bkgnd_dir,'*.json')))

if len(frap_files) != len(bkgnd_files):
    print("ERROR: must have equal number of FRAP and background files")

# Read data from the provided source files
frap_data = [pt.read_trackmate_file(f) for f in frap_files]
bkgnd_data = [frap.read_ij_intensity(f) for f in bkgnd_files]


In [3]:
# Analyze and plot the foci frap data
# Plotting and figure saving params
plot_figs = False
save_figs = True
save_dir = '../reports/figures/IRE1_foci_FRAP'

# Input imaging parameters:
frame_interval = 5 # in seconds #0.2
bleach_n_frames = 2 # how many frames bleaching takes

# For plotting results and keeping track of fits
frap_fits_foci = []

# interactive plotting
if plot_figs: 
    %matplotlib

# create save figure dirs
if save_figs:
    save_dir_png = os.path.join(save_dir, 'png')
    save_dir_svg = os.path.join(save_dir, 'svg')
    if not os.path.exists(save_dir_png):
        os.makedirs(save_dir_png)
    if not os.path.exists(save_dir_svg):
        os.makedirs(save_dir_svg)

# Normalize and fit each pair of frap/background traces
for file, frap_trace, bkgnd_trace in zip(frap_files, frap_data, bkgnd_data):
    # Parse the data into tracks, creating lists of spots that
    # make up each individual track 
    raw_tracks = pt.get_raw_tracks(frap_trace)
    spots = pt.list_spots(frap_trace)
    processed_tracks = pt.build_spot_lists_by_track(raw_tracks, spots)
    track_coords, track_edges, track_spot_ids = processed_tracks

    # Get intensity values for each frame of each track
    all_track_intensities = pt.pull_property_by_track('MEAN_INTENSITY',
                                                   track_spot_ids, spots)

    # Check that spot radii are constant (otherwise comparison of 
    # mean intensities is meaningless)
    all_spot_radii = pt.pull_property_by_track('RADIUS',
                                                track_spot_ids, spots)
    radii_norm = np.ravel(all_spot_radii - all_spot_radii[0])
    diff_radii = np.count_nonzero(radii_norm)
    if diff_radii > 0: print('WARNING: spot sizes are not uniform')

    (bkgnd_int, bkgnd_corr_int) = frap.correct_background (bkgnd_trace, 
                                    track_coords, all_track_intensities)

    # Fit corrected intensities to FRAP recovery curves
    fit, data = frap.fit_frap(bkgnd_corr_int, frame_interval, bleach_n_frames)
    
    # Plot results
    file_name = os.path.basename(file)
    file_name = os.path.splitext(file_name)[0]
    for f,d in zip(fit, data):
        frap_fits_foci.append(f)
        fig, axarr = frap.plot_fit_results(f,d)
        fig.canvas.set_window_title(file_name)
        plt.suptitle(file_name)
        if save_figs:
            fig_filename_svg = os.path.join(save_dir_svg, (file_name+'.svg'))
            fig_filename_png = os.path.join(save_dir_png, (file_name+'.png'))
            plt.savefig(fig_filename_svg)
            plt.savefig(fig_filename_png)


In [4]:
# Load ER FRAP data
import glob

# Path to raw FRAP data (TrackMate xml) and background data (json)
data_dir = '../data/raw/IRE1_ER_FRAP'
frap_subdir = 'FRAP_traces'
bkgnd_subdir = 'bkgnd_traces'

frap_dir = os.path.join(data_dir,frap_subdir)
bkgnd_dir = os.path.join(data_dir,bkgnd_subdir)

frap_files = sorted(glob.glob(os.path.join(frap_dir,'*.json')))
bkgnd_files = sorted(glob.glob(os.path.join(bkgnd_dir,'*.json')))

if len(frap_files) != len(bkgnd_files):
    print("ERROR: must have equal number of FRAP and background files")

# Read data from the provided source files
frap_data = [frap.read_ij_intensity(f) for f in frap_files]
bkgnd_data = [frap.read_ij_intensity(f) for f in bkgnd_files]


In [5]:
# Analyze and plot the ER frap data
# Plotting and figure saving params
save_dir = '../reports/figures/IRE1_ER_FRAP'

# Input imaging parameters:
frame_interval = 5 # in seconds #0.2
bleach_n_frames = 2 # how many frames bleaching takes

# For plotting results and keeping track of fits
frap_fits_er = []

# interactive plotting
if plot_figs: 
    %matplotlib

# create save figure dirs
if save_figs:
    save_dir_png = os.path.join(save_dir, 'png')
    save_dir_svg = os.path.join(save_dir, 'svg')
    if not os.path.exists(save_dir_png):
        os.makedirs(save_dir_png)
    if not os.path.exists(save_dir_svg):
        os.makedirs(save_dir_svg)

# Normalize and fit each pair of frap/background traces
for file, frap_trace, bkgnd_trace in zip(frap_files, frap_data, bkgnd_data):
    
    # subtract background from traces
    bkgnd_int = bkgnd_trace['intensities']
    bkgnd_corr_int = [np.array(frap_trace['intensities']) - np.array(bkgnd_int)]

    
    # Fit corrected intensities to FRAP recovery curves
    fit, data = frap.fit_frap(bkgnd_corr_int, frame_interval, bleach_n_frames)
    
    # Plot results
    file_name = os.path.basename(file)
    file_name = os.path.splitext(file_name)[0]
    for f,d in zip(fit, data):
        frap_fits_er.append(f)
        fig, axarr = frap.plot_fit_results(f,d)
        fig.canvas.set_window_title(file_name)
        plt.suptitle(file_name)
        if save_figs:
            fig_filename_svg = os.path.join(save_dir_svg, (file_name+'.svg'))
            fig_filename_png = os.path.join(save_dir_png, (file_name+'.png'))
            plt.savefig(fig_filename_svg)
            plt.savefig(fig_filename_png)


In [6]:
# Summarize fit results
save_dir = '../reports/figures/IRE1_ER_vs_foci_FRAP_summary'
# create save figure dirs
if save_figs:
    save_dir_png = os.path.join(save_dir, 'png')
    save_dir_svg = os.path.join(save_dir, 'svg')
    if not os.path.exists(save_dir_png):
        os.makedirs(save_dir_png)
    if not os.path.exists(save_dir_svg):
        os.makedirs(save_dir_svg)

%matplotlib

thalf_foci = [f['thalf'] for f in frap_fits_foci]

mobile_f_foci = [f['mobile_fraction'] for f in frap_fits_foci]   
thalf_er = [f['thalf'] for f in frap_fits_er]
mobile_f_er = [f['mobile_fraction'] for f in frap_fits_er]   

# estimate diffusion constant (from doi:10.1111/tra.12008)
r_foci = np.mean(all_spot_radii[0]) # they all have to be the same
roi_areas_er = [x['roi_info']['roi_area'] for x in frap_data]
roi_r_er = [np.sqrt(a/np.pi) for a in roi_areas_er]
d_foci = [0.25*r_foci**2/t for t in thalf_er]
d_er = [0.25*r**2/t for t,r in zip(thalf_er, roi_r_er)]

#print(np.mean(thalf_foci), np.mean(mobile_f_foci))
#print(np.mean(thalf_er), np.mean(mobile_f_er))
dc = [d_er, d_foci]
thalf = [thalf_er, thalf_foci]
mobile = [mobile_f_er, mobile_f_foci]

labels = ('ER lumen', 'Foci')

plt.rcParams["figure.figsize"] = [20,8]
f2, axarr = plt.subplots(1,3, sharey=False)
axarr[0].set_title('Half-time of recovery')
axarr[0].set_ylabel('Time (s)')
axarr[1].set_title('Diffusion constant (estimate)')
axarr[1].set_ylabel(r'D ($\mu m^2/s$)')
axarr[2].set_ylabel('Mobile fraction')
axarr[2].set_title('IRE1 Mobile fraction')
axarr[0].boxplot(thalf, labels=labels)
axarr[1].boxplot(dc, labels=labels)
axarr[2].boxplot(mobile, labels=labels)

if save_figs:
    fig_filename_svg = os.path.join(save_dir_svg, 'Foci_vs_ER_IRE1_recovery.svg')
    fig_filename_png = os.path.join(save_dir_png, 'Foci_vs_ER_IRE1_recovery.png')
    plt.savefig(fig_filename_svg)
    plt.savefig(fig_filename_png)

plt.show()

Using matplotlib backend: MacOSX
